# Max Flow Applications

## Movie distribution

Now suppose a movie distributor would like to ship a copy of a film from CA to every other state. There are therefore 48 units to ship out of CA, and each other state receives 1 unit. 

The dataset contiguous-usa.dat lists the adjacent states in the US. Each line lists two adjacent states; thus AK and HI are omitted, but DC is included in the data. The following code reads in the graph of US states.

In [14]:
import networkx as nx
G = nx.Graph()

usa = open('contiguous-usa.dat')
for line in usa:
    s1, s2 = line.strip().split()
    G.add_edge(s1, s2)

We now encode the demands into the graph.

In [15]:
for state in G.nodes():
    if state != 'CA':
        G.node[state]['demand'] = 1
G.node['CA']['demand'] = -48

We will assign a uniform capacity of 16 to each edge. Since CA has only three adjacent states, this is the smallest possible uniform capacity that allows one to ship all 48 units out of CA. As we have created an undirected graph, and flows have directions, we first convert the graph to a directed graph.

In [16]:
G = nx.DiGraph(G)
uniform_capacity = 16
for (s1, s2) in G.edges():
    G.edge[s1][s2]['capacity'] = uniform_capacity

Complete the following function to implement your algorithm to find a flow with demands. Your function should work correctly for any input, not just the movie instance considered here. As always, you are encouraged to define auxiliary functions as needed for clarity.

In [ ]:
def fold_fulkerson(graph):
    

In [17]:
def flow_with_demands(graph):
    for state in graph.nodes():
        if graph.node[state]['demand'] < 0:
            graph.add_edge('S', state, capacity = -graph.node[state]['demand'])
    
        elif graph.node[state]['demand'] > 0:
            graph.add_edge(state, 'T', capacity = graph.node[state]['demand'])
        
    flow_value, flow_dict = nx.maximum_flow(graph, 'S', 'T')
    
    for state in graph.nodes():
        if state !='T' and state !='S':
            if graph.node[state]['demand'] < 0:
                graph.remove_edge('S', state)
            elif graph.node[state]['demand'] > 0:
                graph.remove_edge(state, 'T')
    
    graph.remove_node('S')
    graph.remove_node('T')
    del flow_dict['S']
    del flow_dict['T']
    
    for i in flow_dict:
        for j in flow_dict[i].keys():
            if j =='S' or j =='T':
                del flow_dict[i][j]
                
    # checking if flow satisfies demands
    s_demand = 0
    t_demand = 0
    for state in G.nodes():
        if  G.node[state]['demand'] < 0:
            s_demand = s_demand+G.node[state]['demand']   
        elif G.node[state]['demand'] > 0:
            t_demand = t_demand+G.node[state]['demand']    
    
    if s_demand + t_demand == 0 and flow_value == t_demand:
        return flow_dict
    else:
        raise nx.NetworkXUnfeasible('No flow satisfying the demands.')

To verify that your solution is correct, implement a function that computes the total flow into each node (which will be negative for supply nodes).

In [18]:
def divergence(flow):
    div_dict = {}
    for i in flow.keys():
        flow_in = 0
        flow_out = 0
        flow_out = sum(flow[i].values())
        for j in flow.keys():
            for k in flow[j]:
                if k == i:
                    flow_in = flow_in + flow[j][k]
        d = flow_in - flow_out
        div_dict[i] = d
    return div_dict

The following code performs a sanity check on your function (but does not completely confirm correctness).

In [19]:
flow = flow_with_demands(G)
div = divergence(flow)
print "Flow satisfies all demands:", all(div[n] == G.node[n]['demand'] for n in G.nodes())

Flow satisfies all demands: True
